In [3]:
import numpy as np
import pydicom as dicom
import os
import matplotlib.pyplot as plt
from glob import glob

In [4]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [5]:
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

In [ ]:
data_path = 'F:/University/S0000001119/S0000001119/'
output_path = working_path = 'F:/University/'
g = glob(data_path + '/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print '\n'.join(g[:5])